In [7]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy 
import requests
from config import weather_api_key
from datetime import datetime
import time 

In [8]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [9]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)


In [10]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

729

In [11]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=8b1d9bbd318414cf754c55c33013aed0


In [12]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [13]:
# Loop through all the cities in the list.
for i, city in enumerate(cities): 

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1 
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_description = str(city_weather["weather"][0]["description"]) 
        city_max_temp = city_weather["main"]["temp_max"] 
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"] 
    
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Current Description": city_description,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")  

Processing Record 1 of Set 1 | hofn
Processing Record 2 of Set 1 | ribeira grande
Processing Record 3 of Set 1 | thompson
Processing Record 4 of Set 1 | belushya guba
City not found. Skipping...
Processing Record 5 of Set 1 | wawa
Processing Record 6 of Set 1 | illoqqortoormiut
City not found. Skipping...
Processing Record 7 of Set 1 | puerto ayora
Processing Record 8 of Set 1 | victoria
Processing Record 9 of Set 1 | yarada
Processing Record 10 of Set 1 | mys shmidta
City not found. Skipping...
Processing Record 11 of Set 1 | saint-francois
Processing Record 12 of Set 1 | dzhuryn
Processing Record 13 of Set 1 | barrow
Processing Record 14 of Set 1 | ushuaia
Processing Record 15 of Set 1 | saldanha
Processing Record 16 of Set 1 | severo-kurilsk
Processing Record 17 of Set 1 | santa cruz de tenerife
Processing Record 18 of Set 1 | san policarpo
Processing Record 19 of Set 1 | jamestown
Processing Record 20 of Set 1 | turukhansk
Processing Record 21 of Set 1 | yuanping
Processing Record 

In [14]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Current Description,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hofn,64.2539,-15.2082,clear sky,44.17,72,7,8.81,IS,2022-04-18 01:59:14
1,Ribeira Grande,38.5167,-28.7000,broken clouds,58.03,69,83,20.85,PT,2022-04-18 01:59:14
2,Thompson,55.7435,-97.8558,broken clouds,32.16,32,75,2.30,CA,2022-04-18 01:59:14
3,Wawa,9.9022,4.4192,overcast clouds,81.36,66,100,7.36,NG,2022-04-18 01:59:15
4,Puerto Ayora,-0.7393,-90.3518,light rain,75.16,95,100,7.20,EC,2022-04-18 01:59:15
5,Victoria,22.2855,114.1577,overcast clouds,74.17,76,99,5.99,HK,2022-04-18 01:57:01
6,Yarada,17.6500,83.2667,broken clouds,84.52,84,75,4.61,IN,2022-04-18 01:59:16
7,Saint-Francois,46.4154,3.9054,clear sky,41.41,86,0,3.44,FR,2022-04-18 01:59:16
8,Dzhuryn,49.0329,25.5839,broken clouds,30.13,79,60,5.21,UA,2022-04-18 01:59:17
9,Barrow,71.2906,-156.7887,clear sky,14.83,83,4,8.99,US,2022-04-18 01:55:01


In [18]:
# Create the output file (CSV).
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")

In [19]:
city_data_reordered = ['City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed', 'Current Description']
city_data_df = city_data_df[city_data_reordered] 
city_data_df.head(10)   

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Hofn,IS,64.2539,-15.2082,44.17,72,7,8.81,clear sky
1,Ribeira Grande,PT,38.5167,-28.7000,58.03,69,83,20.85,broken clouds
2,Thompson,CA,55.7435,-97.8558,32.16,32,75,2.30,broken clouds
3,Wawa,NG,9.9022,4.4192,81.36,66,100,7.36,overcast clouds
4,Puerto Ayora,EC,-0.7393,-90.3518,75.16,95,100,7.20,light rain
5,Victoria,HK,22.2855,114.1577,74.17,76,99,5.99,overcast clouds
6,Yarada,IN,17.6500,83.2667,84.52,84,75,4.61,broken clouds
7,Saint-Francois,FR,46.4154,3.9054,41.41,86,0,3.44,clear sky
8,Dzhuryn,UA,49.0329,25.5839,30.13,79,60,5.21,broken clouds
9,Barrow,US,71.2906,-156.7887,14.83,83,4,8.99,clear sky


In [20]:
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")